In [ ]:
# TODO
# Logic to build job template in AAP

In [ ]:
import requests

In [ ]:
AAP_URL = "https://your-aap.example.com"
USERNAME = "admin"
PASSWORD = "your_password"

TEMPLATE_NAME = "My Example Job Template"
PROJECT_ID = 1              # ID of the project in AAP
INVENTORY_ID = 1            # ID of the inventory in AAP
PLAYBOOK = "remediation.yaml"      # Playbook file in your project
CREDENTIAL_ID = 1           # Credential ID for SSH/machine or vault

In [ ]:
# Disable SSL warnings for self-signed certs (not for production)
requests.packages.urllib3.disable_warnings()

In [ ]:
# --- Authenticate ---
session = requests.Session()
session.auth = (USERNAME, PASSWORD)
session.verify = False  # Change to True if using valid SSL certs
session.headers.update({'Content-Type': 'application/json'})

In [ ]:
# --- Create Job Template ---
url = f"{AAP_URL}/api/v2/job_templates/"

payload = {
    "name": TEMPLATE_NAME,
    "job_type": "run",
    "inventory": INVENTORY_ID,
    "project": PROJECT_ID,
    "playbook": PLAYBOOK,
    "credentials": [CREDENTIAL_ID],
    "ask_verbosity_on_launch": False,
    "ask_variables_on_launch": True,
    "become_enabled": True
}

response = session.post(url, json=payload)

In [ ]:
if response.status_code == 201:
    print(f"✅ Job template '{TEMPLATE_NAME}' created successfully.")
else:
    print(f"❌ Failed to create job template: {response.status_code}")
    print(response.json())